<a href="https://colab.research.google.com/github/609harsh/BTP/blob/main/SQLIADS_Bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'

In [ ]:
!kaggle datasets download -d gambleryu/biggest-sql-injection-dataset -f clean_sql_dataset.csv

 41% 9.00M/21.9M [00:00<00:01, 10.6MB/s]
100% 21.9M/21.9M [00:00<00:00, 27.4MB/s]


In [ ]:
import pandas as pd
sql_dataset = 'clean_sql_dataset.csv.zip'

In [ ]:
dataset = pd.read_csv(sql_dataset)
dataset

,Query,Label
0,""" or pg_sleep ( __TIME__ ) --",1
1,create user name identified by pass123 tempora...,1
2,AND 1 = utl_inaddr.get_host_address ( ...,1
3,select * from users where id = '1' or @ @1 ...,1
4,"select * from users where id = 1 or 1#"" ( ...",1
...,...,...
148321,5555555555555555555555555555555555555555555555...,0
148322,0521094088709893969415119606172279713007042396...,0
148323,,0
148324,"""or 1=1",1


In [ ]:
import re

# Remove non-alphanumeric characters and extra whitespace
def clean_sql_query(query):
    query = re.sub(r'[^a-zA-Z0-9\s]', '', query)
    query = ' '.join(query.split())
    return query

# Apply the cleaning function to your dataset
dataset['Query'] = dataset['Query'].apply(clean_sql_query)


In [ ]:
dataset

,Query,Label
0,or pgsleep TIME,1
1,create user name identified by pass123 tempora...,1
2,AND 1 utlinaddrgethostaddress SELECT DISTINCT ...,1
3,select from users where id 1 or 1 1 union sele...,1
4,select from users where id 1 or 1 union select...,1
...,...,...
148321,5555555555555555555555555555555555555555555555...,0
148322,0521094088709893969415119606172279713007042396...,0
148323,,0
148324,or 11,1


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the SQL queries
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['Query'])

# Convert queries to sequences of integers
sequences = tokenizer.texts_to_sequences(dataset['Query'])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 100  # Choose an appropriate maximum sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [ ]:
from tensorflow.keras.utils import to_categorical

labels = to_categorical(dataset['Label'])


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
1855/1855 [==============================] - 2498s 1s/step - loss: 0.1182 - accuracy: 0.9660 - val_loss: 0.0807 - val_accuracy: 0.9782
Epoch 2/10
1855/1855 [==============================] - 2472s 1s/step - loss: 0.0508 - accuracy: 0.9848 - val_loss: 0.0779 - val_accuracy: 0.9785
Epoch 3/10
1855/1855 [==============================] - 2481s 1s/step - loss: 0.0260 - accuracy: 0.9917 - val_loss: 0.0992 - val_accuracy: 0.9717
Epoch 4/10
1855/1855 [==============================] - 2489s 1s/step - loss: 0.0099 - accuracy: 0.9968 - val_loss: 0.1434 - val_accuracy: 0.9702
Epoch 5/10
1855/1855 [==============================] - 2482s 1s/step - loss: 0.0059 - accuracy: 0.9982 - val_loss: 0.1681 - val_accuracy: 0.9663
Epoch 6/10
1855/1855 [==============================] - 2468s 1s/step - loss: 0.0041 - accuracy: 0.9987 - val_loss: 0.2051 - val_accuracy: 0.9624
Epoch 7/10
1855/1855 [==============================] - 2466s 1s/step - loss: 0.0036 - accuracy: 0.9988 - val_loss: 0.2121 -

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))

optimizer = Adam(lr=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
1855/1855 [==============================] - 1115s 600ms/step - loss: 0.1235 - accuracy: 0.9642 - val_loss: 0.0831 - val_accuracy: 0.9766
Epoch 2/10
1855/1855 [==============================] - 1112s 600ms/step - loss: 0.0578 - accuracy: 0.9831 - val_loss: 0.0798 - val_accuracy: 0.9753
Epoch 3/10
1855/1855 [==============================] - 1109s 598ms/step - loss: 0.0365 - accuracy: 0.9896 - val_loss: 0.0922 - val_accuracy: 0.9731
Epoch 4/10
1855/1855 [==============================] - 1106s 596ms/step - loss: 0.0211 - accuracy: 0.9927 - val_loss: 0.1249 - val_accuracy: 0.9623
Epoch 5/10
1855/1855 [==============================] - 1107s 597ms/step - loss: 0.0107 - accuracy: 0.9964 - val_loss: 0.1283 - val_accuracy: 0.9680
Epoch 6/10
1855/1855 [==============================] - 1101s 594ms/step - loss: 0.0062 - accuracy: 0.9979 - val_loss: 0.1701 - val_accuracy: 0.9680
Epoch 7/10
1855/1855 [==============================] - 1110s 599ms/step - loss: 0.0052 - accuracy: 0.9984

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Function to calculate confusion matrix
def get_confusion_matrix(y_true, y_pred):
    y_true = tf.argmax(y_true, axis=1)
    y_pred = tf.argmax(y_pred, axis=1)
    return tf.math.confusion_matrix(y_true, y_pred)

# Your original model definition
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.F1Score(), get_confusion_matrix])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)


Epoch 1/10
1855/1855 [==============================] - 1271s 684ms/step - loss: 0.2414 - accuracy: 0.9156 - precision: 0.9156 - recall: 0.9156 - f1_score: 0.9156 - get_confusion_matrix: 15.9919 - val_loss: 0.1388 - val_accuracy: 0.9618 - val_precision: 0.9618 - val_recall: 0.9618 - val_f1_score: 0.9618 - val_get_confusion_matrix: 15.9838
Epoch 2/10
1855/1855 [==============================] - 1255s 677ms/step - loss: 0.1120 - accuracy: 0.9686 - precision: 0.9686 - recall: 0.9686 - f1_score: 0.9686 - get_confusion_matrix: 15.9919 - val_loss: 0.1088 - val_accuracy: 0.9702 - val_precision: 0.9702 - val_recall: 0.9702 - val_f1_score: 0.9702 - val_get_confusion_matrix: 15.9838
Epoch 3/10
1855/1855 [==============================] - 1275s 687ms/step - loss: 0.0858 - accuracy: 0.9760 - precision: 0.9760 - recall: 0.9760 - f1_score: 0.9760 - get_confusion_matrix: 15.9919 - val_loss: 0.1007 - val_accuracy: 0.9731 - val_precision: 0.9731 - val_recall: 0.9731 - val_f1_score: 0.9731 - val_get_con